In [46]:
import psycopg2
import os
import re
import pandas as pd
from langchain.tools import tool
from urllib.parse import quote_plus
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [48]:
prompt = ChatPromptTemplate(
    [
        ("system",custom_prompt_str),
        ("user","{input}" )
    ]
)
print(prompt.input_variables)
print(prompt.output_parser)

['input']
None


In [50]:
def extract_sql_from_text(text: str) -> str:
    """
    Extracts the first SQL code block from the LLM response.
    """
    match = re.search(r"```sql\n(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # Fallback: return everything from SELECT onward
        select_pos = text.lower().find("select")
        return text[select_pos:].strip() if select_pos != -1 else text

In [51]:
@tool
def executeQuery(query: str):
    '''
    This function used to fetch all data based on the query
    '''
    final_query = extract_sql_from_text(query)
    conn = createSupabaseConnection()
    print("Query from executeQuery -->", query)
    df = pd.read_sql(final_query, conn)
    conn.close()
    #return df
    return df.to_markdown(index=False, tablefmt="retaildata")

In [52]:
tools = [executeQuery]
prompt_hub = hub.pull("hwchase17/react")

In [54]:
response = prompt | llm
output=response.invoke({"input":{user_query}})
print(output.content)
print("from extract_sql_from_text \n",extract_sql_from_text(output.content))
query_output = extract_sql_from_text(output.content)

```sql
SELECT 
    fol.order_id,
    fol.order_placement_date,
    fol.customer_id,
    fol.product_id,
    fol.order_qty,
    fol.agreed_delivery_date,
    fol.actual_delivery_date,
    fol.delivery_qty,
    fol."In Full",
    fol."On Time",
    fol."On Time In Full"
FROM 
    fact_order_line fol
WHERE 
    fol."In Full" = 'Yes' AND fol."On Time" = 'Yes';
```
from extract_sql_from_text 
 SELECT 
    fol.order_id,
    fol.order_placement_date,
    fol.customer_id,
    fol.product_id,
    fol.order_qty,
    fol.agreed_delivery_date,
    fol.actual_delivery_date,
    fol.delivery_qty,
    fol."In Full",
    fol."On Time",
    fol."On Time In Full"
FROM 
    fact_order_line fol
WHERE 
    fol."In Full" = 'Yes' AND fol."On Time" = 'Yes';


In [55]:
agent = create_react_agent(
    llm=llm,
    tools = tools,
    prompt= prompt_hub
)

agent_executor = AgentExecutor(
    agent = agent,
    tools= tools,
    verbose=False
)


In [56]:
final_input = f"""
User Question: {user_query}
Query to Execute: {output.content}
"""

In [57]:
print(query_output)
response = agent_executor.invoke({"input":final_input})
print(agent_executor.input_keys)
print(response["output"])

SELECT 
    fol.order_id,
    fol.order_placement_date,
    fol.customer_id,
    fol.product_id,
    fol.order_qty,
    fol.agreed_delivery_date,
    fol.actual_delivery_date,
    fol.delivery_qty,
    fol."In Full",
    fol."On Time",
    fol."On Time In Full"
FROM 
    fact_order_line fol
WHERE 
    fol."In Full" = 'Yes' AND fol."On Time" = 'Yes';
conn --> <connection object at 0x119f9f760; dsn: 'user=postgres.fxpuwjgwfhkapgjinyss password=xxx dbname=postgres host=aws-0-us-east-2.pooler.supabase.com port=5432 sslmode=require', closed: 0>
Query from executeQuery --> ```sql
SELECT 
    fol.order_id,
    fol.order_placement_date,
    fol.customer_id,
    fol.product_id,
    fol.order_qty,
    fol.agreed_delivery_date,
    fol.actual_delivery_date,
    fol.delivery_qty,
    fol."In Full",
    fol."On Time",
    fol."On Time In Full"
FROM 
    fact_order_line fol
WHERE 
    fol."In Full" = 'Yes' AND fol."On Time" = 'Yes';
```



/var/folders/7_/vzxv0s1j43n3s8qd35q3b6_m0000gn/T/ipykernel_79965/2250908431.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(final_query, conn)


[]
There are no orders in the database that were shipped both in full and on time.
